## AI Model

Predicting based on historical records.

In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset
df = pd.read_csv("shipping_dataset.csv")

# Select predictors and target variables
X_cols = ["week", "time", "port", "demand", "temperature", "humidity", "market_trend", "day"]
y_col_containers = 'containers'
y_col_ships = 'ships'
y_col_operational_cost = 'operational_cost'

# Define a custom encoding function for market_trend
def encode_market_trend(value):
    if value == "negative":
        return -1
    elif value == "neutral":
        return 0
    elif value == "positive":
        return 1
    else:
        return 0  # Default to 0 for unknown values

# Apply the custom encoding to the market_trend column
df['market_trend'] = df['market_trend'].apply(encode_market_trend)

# Define transformers for encoding categorical variables (week, and day)
categorical_features = ["week", "day"]
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Combine transformers for preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, categorical_features)
    ])

# Create and train the linear regression models
model_containers = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model_containers.fit(df[X_cols], df[y_col_containers])

model_ships = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model_ships.fit(df[X_cols], df[y_col_ships])

model_operational_cost = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])

model_operational_cost.fit(df[X_cols], df[y_col_operational_cost])

print("Model is successfully built!")

Model is successfully built!


## Adding record to database

In [2]:
def addRecord():
    import csv, datetime
    
    def enter(text):
        while True:
            try:
                ans = float(input(text))
                break
            except:
                print("Invalid Value")         
        return ans

    # Collect data for the new record
    containers = enter("How many shipping containers?")
    ships = enter("How many cargo ships?")
    port = enter("Which port? (1 or 2)")
    operational_cost = enter("Estimated operational cost")
    demand = enter("Demand (0 - 300)")
    temperature = enter("Temperature")
    humidity = enter("Humidity")
    market_trend = input("Market trend (positive/neutral/negative): ")

    # Define the new record as a dictionary
    new_record = {
        "year": datetime.datetime.now().year,
        "week": datetime.datetime.now().strftime("%U"),
        "day": datetime.datetime.now().strftime("%a"),
        "time": datetime.datetime.now().strftime("%H:%M"),
        "containers": containers,
        "ships": ships,
        "port": int(port),
        "operational_cost": operational_cost,
        "demand": demand,
        "temperature": temperature,
        "humidity": humidity,
        "market_trend": market_trend.lower()
    }

    # Specify the CSV file path
    file_path = "shipping_dataset.csv"

    # Open the CSV file in append mode and write the new record
    with open(file_path, mode="a", newline="") as file:
        writer = csv.DictWriter(file, fieldnames=new_record.keys())

        # If the file is empty, write the header row
        if file.tell() == 0:
            writer.writeheader()

        # Write the new record
        writer.writerow(new_record)

    print("New record added successfully.")

## Management System

In [3]:
def main():
    import datetime
    print("Welcome to the Database Management System")
    
    while True:
        print("\nOptions:")
        print("1. Add a Record")
        print("2. Forecast Traffic")
        print("3. Exit")
        choice = input("Enter your choice: ")

        if choice == "1":
            addRecord()
            
        elif choice == "2":
            # Ask the user for a date in DD/MM/YYYY format
            date_input = input("Enter date (DD/MM/YYYY): ")

            try:
                # Parse the date to extract components
                day, month, year = map(int, date_input.split('/'))
                date = datetime.date(year, month, day)
                week = date.strftime("%U")
            except ValueError:
                print("Invalid date format. Please use DD/MM/YYYY.")
                continue

            # Ask the user for the remaining predictors
            port = input("Enter port (1 or 2): ")
            demand = input("Enter demand (0 - 300): ")
            temperature = input("Enter temperature: ")
            humidity = input("Enter humidity: ")
            market_trend = input("Enter market trend (positive/neutral/negative): ")

            # Create a dictionary of predictors
            predictors = {
                "week": week,
                "time": datetime.datetime.now().strftime("%H:%M"),
                "port": port,
                "demand": demand,
                "temperature": temperature,
                "humidity": humidity,
                "market_trend": market_trend.lower(),
                "day": date.strftime("%a"),
            }

            # Convert the dictionary into a DataFrame
            predictors_df = pd.DataFrame([predictors], columns=X_cols)

            # Use all three models to make predictions
            containers_pred = model_containers.predict(predictors_df)
            ships_pred = model_ships.predict(predictors_df)
            operational_cost_pred = model_operational_cost.predict(predictors_df)

            # Print out the predictions
            print(f"Predicted Containers: {containers_pred[0]}")
            print(f"Predicted Ships: {ships_pred[0]}")
            print(f"Predicted Operational Cost: {operational_cost_pred[0]}")

        elif choice == "3":
            print("Goodbye.")
            break
            
        else:
            print("Invalid choice. Please select a valid option.")

if __name__ == "__main__":
    main()

Welcome to the Database Management System

Options:
1. Add a Record
2. Forecast Traffic
3. Exit
Enter your choice: 1
How many shipping containers?300
How many cargo ships?1
Which port? (1 or 2)1
Estimated operational cost2000
Demand (0 - 300)100
Temperature32
Humidity40
Market trend (positive/neutral/negative): positive
New record added successfully.

Options:
1. Add a Record
2. Forecast Traffic
3. Exit
Enter your choice: 2
Enter date (DD/MM/YYYY): 1/10/2023
Enter port (1 or 2): 1
Enter demand (0 - 300): 100
Enter temperature: 30
Enter humidity: 40
Enter market trend (positive/neutral/negative): neutral
Predicted Containers: 1181094.6431434308
Predicted Ships: 5652.425793927783
Predicted Operational Cost: 6319196.49860245

Options:
1. Add a Record
2. Forecast Traffic
3. Exit
Enter your choice: 3
Goodbye.
